<div class="head0">
    <div class="head0__name">
        Comparison of solvers
    </div>
    <div class="head0__note">
        Comparison of 1th order and 2th order solvers.
    </div>
</div>

In [1]:
import sys
import time

import numpy as np
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt

sys.path.extend(['./../lib', './../helpers'])
from intertrain import Intertrain
from solver import Solver
from solver_tmp import SolverTmp
from config import config

from helpers import init_jupyter; init_jupyter()

Start |  5:29PM MSK on Sep 03, 2019 |
-------------------------------------


<div class="head1">
    <div class="head1__name">
        Parameters
    </div>
</div>

In [2]:
A = 1.
s = 1.
Dc = 0.5

def func_f0(x, t):
    m = 0.
    return A * (m - x)

def func_f1(x, t):
    return -A * np.ones(x.shape)

def func_xm(t, x0):
    return np.exp(-A * t) * x0
 
def func_xc(t):
    return (1. - np.exp(-2. * A * t)) / 2. / A

def func_r0(x):
    m = 0.
    r = np.exp(-0.5 * (x - m)**2 / s) / np.sqrt(2. * np.pi * s)
    return r.reshape(-1)

def func_rx(x, t, x0):
    m = func_xm(t, x0)
    c = func_xc(t)
    r = np.exp(-0.5 * (x - m)**2 / c) / np.sqrt(2. * np.pi * c)
    return r.reshape(-1)

def func_rt(x, t):
    S = func_xc(t) + s * np.exp(-2. * A * t)
    r = np.exp(-0.5 * x**2 / S) / np.sqrt(2. * np.pi * S)
    return r.reshape(-1)

def func_rs(x):
    r = np.sqrt(A / np.pi) * np.exp(- A * x * x)
    return r.reshape(-1)

In [3]:
t_poi = 1000
t_min = 0.
t_max = 1.

x_poi = 50
x_min = -5.
x_max = +5.

m = t_poi
n = x_poi
l = (x_max - x_min) / x_poi
h = (t_max - t_min) / (t_poi - 1)
T = np.linspace(t_min, t_max, t_poi)

print('Time-space factor (h/l^2) : %-8.5f'%(h/l/l))

Time-space factor (h/l^2) : 0.02503 


<div class="head1">
    <div class="head1__name">
        Computations
    </div>
</div>

In [4]:
print('1th order solver: ')
time.sleep(1.)
SL = Solver(d=1)
SL.set_grid_t(m, t_min, t_max, t_hst=1)
SL.set_grid_x(n, x_min, x_max)
SL.set_funcs(func_f0, func_f1, func_r0, func_rt, func_rs)
SL.set_coefs(Dc)
SL.prep()
SL.calc()

print('2th order solver (custom ode solver): ')
time.sleep(1.)
SL = Solver(d=1, ord=2)
SL.set_grid_t(m, t_min, t_max, t_hst=1)
SL.set_grid_x(n, x_min, x_max)
SL.set_funcs(func_f0, func_f1, func_r0, func_rt, func_rs)
SL.set_coefs(Dc)
SL.prep()
SL.calc()

print('2th order solver (standard odeint solver): ')
time.sleep(1.)
SL = SolverTmp(d=1)
SL.set_grid_t(m, t_min, t_max, t_hst=1)
SL.set_grid_x(n, x_min, x_max)
SL.set_funcs(func_f0, func_f1, func_r0, func_rt, func_rs)
SL.set_coefs(Dc)
SL.prep()
SL.calc()

1th order solver: 


Solve: 100%|█| 999/999 [00:01<00:00, 671.69step/s, | At T = 1.00e+00 : error = 5.48e-04]


2th order solver (custom ode solver): 


Solve: 100%|█| 999/999 [00:02<00:00, 468.99step/s, | At T = 1.00e+00 : error = 3.31e-04]


2th order solver (standard odeint solver): 


Solve: 100%|█| 999/999 [00:55<00:00, 16.16step/s, | At T = 1.00e+00 : error = 3.31e-04]


<div class="end"></div>